In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

In [ ]:
# input 320x160, steering angle
# NV paper, input 200x66, YUV
# 200x100
# 320x88

model = Sequential()
model.add(Convolution2D(24, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2), input_shape=(66, 200, 3)))
# model.add(MaxPooling2D(border_mode='same'))
model.add(Convolution2D(36, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2)))
# model.add(MaxPooling2D(border_mode='same'))
model.add(Convolution2D(48, 5, 5, border_mode='valid', activation='relu', subsample=(2, 2)))
# model.add(MaxPooling2D(border_mode='same'))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(Flatten())
# model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile('adam', 'mse')
# model.summary()

In [ ]:
driving_log = pd.read_csv('data/driving_log.csv')
num_of_sample = len(driving_log)


In [ ]:
def preprocessing(img):
    # crop 160x320 ==> 99x300
    img_tmp = img[41:139, 10:310, :]
    # resize 99x300 ==> 66x200
    img_tmp = cv2.resize(img_tmp, (200, 66))
    # BGR2YUV
    img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2YUV)
    # normalize
    img_tmp = img_tmp.astype(np.float32)
    img_tmp = (img_tmp - 128) / 128
    
    return img_tmp

In [ ]:
def generate_arrays_from_file(batch_size=32):
    while True:
        X = []
        y = []

        for i in range(batch_size):
            # select idx
            idx = np.random.randint(0, num_of_sample)
            # select camera, 0 center, 1 left, 2 right
            camera = np.random.randint(0, 3)

            filename = driving_log.iloc[idx][camera].strip()
            steering = driving_log.iloc[idx]['steering']
            if camera == 1:
                steering -= 0.25
            elif camera == 2:
                steering += 0.25
            img = cv2.imread('data/' + filename)
            img = preprocessing(img)
            # select mirror
            if np.random.randint(0, 2) % 2 == 1:
                img = img[:, ::-1, :]
                steering *= -1

            X.append(img)
            y.append(steering)
        assert len(X) == batch_size
        yield np.array(X), np.array(y)

In [ ]:
history = model.fit_generator(generate_arrays_from_file(), 20000, 8)
# model.fit_generator(generate_arrays_from_file('/my_file.txt'),
#         samples_per_epoch=10000, nb_epoch=10)

In [ ]:
json_string = model.to_json()
json.dump(json_string, 'model.json')
model.save_weights('model.h5')

In [ ]:
c2 = preprocessing(cv2.imread('data/IMG/' + CENTER))